In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
from learn import *

In [7]:
learn, denorm, data = learner_on_dataset(datapaths[1], debug=True)
(trn_x,trn_y), (val_x,val_y) = data

HBox(children=(IntProgress(value=0, description='ims', max=2695), HTML(value='')))

HBox(children=(IntProgress(value=0, description='ims', max=1156), HTML(value='')))

NameError: name '_bs' is not defined